[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/038_clasificacion_texto/clasificacion_texto.ipynb)

# Clasificación de texto

En el [post](https://sensioai.com/blog/037_charRNN) anterior aprendimos a cómo podemos entrenar una `red neuronal recurrente` para generar texto letra a letra, una tarea muy interesante dentro del mundo del `procesado de lenguaje natur` o `NLP` (*natural language processing*) por sus siglas en inglés. Aún así, posiblemente la tarea más interesante desde un punto de vista práctico y con más aplicaciones en la industria sea la de `clasificación de texto`. De la misma manera que podemos entrenar `redes neuronales` para [clasificación de imágenes](https://sensioai.com/blog/033_receta_entrenamiento), es posible entrenar modelos de `machine learning` capaces de asignar una etiqueta determinada a un trozo de texto. Podemos encontrar este tipo de aplicaciones en redes sociales, por ejemplo, para detectar automáticamente mensaje ofensivos o en opiniones de usuarios sobre artículos para medir su satisfacción. En este post vamos a ver cómo podemos entrenar una `red neuronal recurrente` para clasificar *reviews* de películas, una tarea también conocida por el nombre de `sentiment analysis`.

 ## El *dataset*

En el post anterior descargamos el libro *Don Quijote de la Mancha* en formato *txt*, luego lo cargamos en `Python` para proceder al proceso de `tokenización`. Si bien implementamos nuestra propia lógica de procesado de texto, a la práctica es más conveniente utilizar herramientas de terceros bien testeadas y optimizadas. Entre las diferentes librerías que existen de `NLP`, nosotros utilizaremos [torchtext](https://pytorch.org/text/) ya que está bien integrada con el ecosistema de `Pytorch`. 

In [2]:
!pip install torchtext==0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.0 which is incompatible.


In [3]:
import torch
import torchtext
from torchtext.legacy import datasets
from torchtext.legacy import data
# from torchtext import datasets
# from torchtext import data

En `torchtext` tenemos disponible multitud de datasets que podemos utilizar, los cuales son ideales cuando estamos aprendiendo a trabajar con `redes neuronales` para tareas de `NLP`. En este caso descargaremos el dataset `IMDB` que contiene opiniones sobre películas. Nuestro objetivo será, dada una de estas *reviews* asignarle una etiqueta binaria (opinión positiva o negativa). 

In [4]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.long)

# fields = {'text': ('t', TEXT), 'label': ('l', LABEL)}

# train_data, valid_data, test_data = TabularDataset.splits(path=path_datasets,
#                                                                train="lcquad10_train.csv",
#                                                                test="lcquad10_test.csv",
#                                                                validation="lcquad10_valid.csv",
#                                                                format="csv",
#                                                                fields=fields)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 33.7MB/s]


> ⚡ La clase `torchtext.data.Field` contiene toda la lógica de tokenización y procesado de texto necesaria, lo cual nos facilitará mucho la vida para esta tarea. Puedes aprender más sobre esta clase, y la librería en general, en su [documentación](https://pytorch.org/text/data.html).

En este caso en concreto, disponemos de 25000 muestras tanto para el entrenamiento como evaluación de nuestros modelos.

In [5]:
len(train_data), len(test_data)

(25000, 25000)

De la siguiente manera podemos ver un ejemplo de muestra de nuestro dataset, que está compuesto por el texto y la valoración.

In [8]:
print(train_data[0:10])
print(vars(train_data.examples[0]))

[<torchtext.legacy.data.example.Example object at 0x7f4c82d30210>, <torchtext.legacy.data.example.Example object at 0x7f4c7de50e10>, <torchtext.legacy.data.example.Example object at 0x7f4c7de5ef90>, <torchtext.legacy.data.example.Example object at 0x7f4c7de6b8d0>, <torchtext.legacy.data.example.Example object at 0x7f4c7e274790>, <torchtext.legacy.data.example.Example object at 0x7f4c7e2a7950>, <torchtext.legacy.data.example.Example object at 0x7f4c7e2ae6d0>, <torchtext.legacy.data.example.Example object at 0x7f4c81a32610>, <torchtext.legacy.data.example.Example object at 0x7f4c81a34350>, <torchtext.legacy.data.example.Example object at 0x7f4c81a4cc10>]
{'text': ['The', 'great', 'talents', 'of', 'Michael', 'Powell', 'and', 'Emeric', 'Pressberger', 'are', 'noticeable', 'in', 'their', 'wonderful', '"', 'A', 'Matter', 'of', 'Life', 'and', 'Death', '"', '.', 'It', 'was', 'part', 'of', 'the', 'recent', 'tribute', 'to', 'Mr.', 'Powell', 'that', 'played', 'at', 'the', 'Walter', 'Reade', 'in', 

### Tokenización

Además de proveernos con varios datasets, `torchtext` también nos facilita mucho la vida a la hora de llevar a cabo el proceso de `tokenización`. En este caso vamos a construir un vocabulario que contendrá un número determinado de palabras (ya que si aqueremos incluirlas los requisitos computacionales se dispararían). Para ello el tokenizador calculará la frecuencia de cada palabra en el texto y se quedará con la cantidad que especifiquemos.

In [9]:
MAX_VOCAB_SIZE = 10000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

len(TEXT.vocab), len(LABEL.vocab)

(10002, 2)

Como pudes ver tenemos un vocabulario con la longitud determinada más dos, estos dos *tokens* extra corresponden a los *tokens* `<unk>`, que se le asignarán a las palabras desconocidas y las menos frecuentes que no hayan pasado el primer filtro, y el *token* `<pad>`, que se usará para que todas las frases en un *batch* tengan la misma longitud (necesitamos tensores recutangulares para entrenar nuestra red).

In [10]:
TEXT.vocab.freqs.most_common(10)
# TEXT.vocab.stoi
# TEXT.vocab.itos

[('the', 289838),
 (',', 275296),
 ('.', 236843),
 ('and', 156483),
 ('a', 156282),
 ('of', 144055),
 ('to', 133886),
 ('is', 109095),
 ('in', 87676),
 ('I', 77546)]

In [11]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

In [12]:
LABEL.vocab.stoi

defaultdict(None, {'neg': 0, 'pos': 1})

El último paso para tener nuestros datos listos para entrenar una red neuronal es construir el `DataLoader` encargado de alimentar nuestra red con *batches* de frases de manera eficiente. Para ello utilizamos la clase `torchtext.data.BucketIterator`, que además juntará frases de similar longitud minimazndo el *padding* necesario.

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

dataloader = {
    'train': data.BucketIterator(train_data, batch_size=128, shuffle=True, sort_within_batch=True, device=device),
    'test': data.BucketIterator(test_data, batch_size=128, device=device)
}

cuda


## El modelo

Para poder clasificar texto utilizaremos una [red recurrente](https://sensioai.com/blog/034_rnn_intro) de tipo `many-to-one`, la cual recibirá el texto palabra por palabra. Usaremos el último estado oculto (el cual contendrá información de toda la frase) para generar nuestra predicción final. Cada palabra estará representada por un vector, el cual será aprendido por la red en la capa `embedding` (puedes aprender más sobre esta capa en nuestro [post](https://sensioai.com/blog/037_charRNN) anterior).

In [14]:
class RNN(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim=128, hidden_dim=128, output_dim=2, num_layers=2, dropout=0.2, bidirectional=False):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.GRU(
            input_size=embedding_dim, 
            hidden_size=hidden_dim, 
            num_layers=num_layers, 
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        self.fc = torch.nn.Linear(2*hidden_dim if bidirectional else hidden_dim, output_dim)
        
    def forward(self, text):
        #text = [sent len, batch size]        
        embedded = self.embedding(text)        
        #embedded = [sent len, batch size, emb dim]        
        output, hidden = self.rnn(embedded)        
        #output = [sent len, batch size, hid dim]
        y = self.fc(output[-1,:,:].squeeze(0))     
        return y

> 💡 A diferencia de los pots anteriores, ahora la dimensión *batch* NO es la primera. Este es el comportamiento por defecto de las capas recurrentes en `Pytorch`. Puedes modificar esto añadiendo la opción `batch_first=True` en la capa recurrente (y asegúrate que tu dataloader utiliza también la primera dimensión para el batch. En `torchtext` puedes indicarlo con el parámetro `batch_first=True` a la hora de definir el `FIELD` en cuestión). 

Como siempre, probamos que nuestra red esté bien definida y las dimensiones encajen. A la entrada, esperamos tensores con dimensiones `longitud secuencia x batch`. Puedes verlo si sacamos unas muestras de nuestro dataloader.

In [15]:
batch = next(iter(dataloader['train']))

batch.text.shape

torch.Size([349, 128])

In [26]:
print(batch.text[:, 127])

tensor([  11,   73,  819,    8,  959,   16,   28,  131, 5971,    3,   26,   19,
         232,   21,    2, 1009,    3,   44,    3,   20,    6, 1014, 3113,    3,
         716, 7824, 6030, 1606,    5, 3318, 1331,   21,    2,  957,   14,    0,
          88,  291,   60,    3,   26,  156,  172,   33,   10,    2, 1355,   23,
          13,   22,    4,   14, 4716,    0,    2,   14,    0,    0, 1192,    0,
          18, 3011,    3,   11,  276,    0,  503,    0,    6,  274,  777,   22,
           4,  555, 1292, 1049, 2725,    3,   12,   85,   34,  410,    7,  277,
           5,    6,  186,  488,   55, 1332,  410,    4,   11,   19, 8623,   21,
           2,  211,   13,   12,   19,    6, 1047,    3,    5,   20,    6, 1014,
        3113,  130,  117,  285,    8, 3085,   13,  211,    5,  130,  109,   28,
          79,  218, 5432, 1077,    4,  764, 7298, 1477,   10,    6, 2041,    5,
        1489, 2103,   20,    6,    0, 2375,    0,   13,    3,   23,    2,  105,
         197,  338,   84, 1570,  127,  2

Cada palabra estrá representada por un índice, que luego el `embedding` usará para extraer el vector determinado que representa la palabra. A la salida, nuestro modelo nos dará dos valores. Si el primer valor es mayor que el segundo, asignaremos la clase `0` (opinión negativa) y viceversa.

In [27]:
model = RNN(input_dim=len(TEXT.vocab))
outputs = model(torch.randint(0, len(TEXT.vocab), (100, 64)))
outputs.shape

torch.Size([64, 2])

## Entrenamiento

Para entrenar nuestra red usamos el bucle estándar que ya usamos en posts anteriores.

In [28]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch.text, batch.label
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch.text, batch.label
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [29]:
fit(model, dataloader)

val_loss 0.71261 val_acc 0.50197: 100%|██████████| 196/196 [00:11<00:00, 17.40it/s]


Epoch 1/5 loss 0.66552 val_loss 0.71261 acc 0.58906 val_acc 0.50197


val_loss 1.11421 val_acc 0.50759: 100%|██████████| 196/196 [00:11<00:00, 17.36it/s]


Epoch 2/5 loss 0.52902 val_loss 1.11421 acc 0.73838 val_acc 0.50759


val_loss 0.36842 val_acc 0.84211: 100%|██████████| 196/196 [00:11<00:00, 17.38it/s]


Epoch 3/5 loss 0.40182 val_loss 0.36842 acc 0.82474 val_acc 0.84211


val_loss 0.31013 val_acc 0.87338: 100%|██████████| 196/196 [00:11<00:00, 17.08it/s]


Epoch 4/5 loss 0.28718 val_loss 0.31013 acc 0.88057 val_acc 0.87338


val_loss 0.27751 val_acc 0.88396: 100%|██████████| 196/196 [00:11<00:00, 17.22it/s]

Epoch 5/5 loss 0.23763 val_loss 0.27751 acc 0.90740 val_acc 0.88396


## Generando predicciones

Ahora ya podemos utilizar nuestro modelo para generar valoraciones de manera automática dada una opinión.

In [30]:
import spacy
nlp = spacy.load('en')

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X)
        return pred

In [31]:
sentences = ["this film is terrible", "this film is great", "this film is good", "a waste of time", "I hate it."]
tokenized = [[tok.text for tok in nlp.tokenizer(sentence)] for sentence in sentences]
indexed = [[TEXT.vocab.stoi[_t] for _t in t] for t in tokenized]
tensor = torch.tensor(indexed).permute(1,0)
predictions = torch.argmax(predict(model, tensor), axis=1)
predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


tensor([0, 1, 1, 0, 0], device='cuda:0')

En este caso solo tenemos dos posibles clases, pero es fácil intuir que de ser capaces de construir un dataset con muchas más clases que describan con mayor precisión el "sentimiento" en un texto podríamos extraer muchísima información muy valiosa para, sobre todo, empresas que venden productos online más allá de las típicas estrellas o puntuaciones que, pese a dar información valiosa, no aportan ningún tipo de información accionable.

## Redes Recurrentes Bidireccionales

Las redes recurrentes bidireccionales nos van a permitir, por norma general, obtener mejores resultados cuando trabajemos con datos secuenciales en los que "podamos mirar al futuro". En aplicaciones tales como la generación de texto o la predicción de series temporales, esto no lo podíamos hacer ya que el objetivo de la tarea es precisamente predecir valores futuros (y utilizar estos valores durante el entrenamiento no tendría sentido). Sin embargo, para la tarea de clasificación de texto, sí que podemos hacerlo.

![](https://miro.medium.com/max/764/1*6QnPUSv_t9BY9Fv8_aLb-Q.png)

Puedes conocer más sobre este tipo de redes, así como otras mejoras, en este [post](https://sensioai.com/blog/036_rnn_mejoras).

In [ ]:
model = RNN(input_dim=len(TEXT.vocab), bidirectional=True)
fit(model, dataloader)

val_loss 0.63835 val_acc 0.52760: 100%|██████████| 196/196 [00:18<00:00, 10.44it/s]


Epoch 1/5 loss 0.65129 val_loss 0.63835 acc 0.60130 val_acc 0.52760


loss 0.47277 acc 0.78142:  23%|██▎       | 45/196 [00:03<00:11, 12.85it/s]

In [ ]:
sentences = ["this film is terrible", "this film is great", "this film is good", "a waste of time", "so good bad film", "so little bad good"]
tokenized = [[tok.text for tok in nlp.tokenizer(sentence)] for sentence in sentences]
indexed = [[TEXT.vocab.stoi[_t] for _t in t] for t in tokenized]
tensor = torch.tensor(indexed).permute(1,0)
predictions = torch.argmax(predict(model, tensor), axis=1)
predictions

## Resumen

En este post hemos aprendido a utilizar `redes neuronales recurrentes` para la tarea de clasificación de texto. Esta tarea es muy útil en la industria, sobre todo para aquellos negocios que venden productos o servicios cuyos usuarios pueden valorar directamente de manera online de forma masiva. Tener un sistema automatizado que "lea" todas las opiniones y las clasifique en clases con significado, puede aportar mucha información valiosa a una empresa sobre la cual puede llevar a cabo acciones de mejora de manera rápida. Como hemos visto, el uso de la librería `torchtext` nos facilita mucho la vida a la hora de procesar el texto, y gracias a su integración con `Pytorch` podremos entrenar modelos de manera rápida y sencilla. Para esta tarea en concreto, también hemos visto que el uso de `redes recurrentes bidireccionales` nos puede dar un extra de precisión.